In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from ipywidgets import interact
import ipywidgets as widgets

import sys
sys.path.insert(0, '../../')
from kelp import Filter, Planet, Model

In [ ]:
# Set initial parameters
n = 'HD 189733'
channel = 1
n_theta = 20
n_phi = 25
f = 1/2**0.5
ln_c11 = 1
ln_c13 = 0
c23 = 0
c33 = 0
ln_omega_drag = 5
alpha = 0.6
hotspot_offset = 0


# Import planet properties, filter properties
p = Planet.from_name(n)
filt = Filter.from_name(f"IRAC {channel}")
filt.bin_down()

# Create interactive plot:
@interact(
    offset=widgets.FloatSlider(min=-2, max=2, step=0.1, value=0),
    ln_c11=widgets.FloatSlider(min=-2, max=2, step=0.1, value=ln_c11, 
                               description='$\log C_{11}$'),
    alpha=widgets.FloatSlider(min=0, max=1, step=0.1, value=alpha,
                              description='$\\alpha$'),
    ln_omega_drag=widgets.FloatSlider(min=0, max=10, step=0.5, value=ln_omega_drag, 
                                      description='$\log \omega_{\\rm drag}$'),
    c11=widgets.FloatSlider(min=-2, max=2, step=0.1, value=ln_c11, 
                            description='$\log C_{11}$'),
    ln_c13=widgets.FloatSlider(min=-2, max=2, step=0.1, value=ln_c13, 
                               description='$\log C_{13}$'),
    c23=widgets.FloatSlider(min=-2, max=0, step=0.1, value=c23, 
                            description='$C_{23}$'),
    c33=widgets.FloatSlider(min=-2, max=0, step=0.1, value=c33, 
                            description='$C_{33}$')
)
def temperature_map_plot(offset, alpha, ln_omega_drag, ln_c11, ln_c13, c23, c33):
    C_ml = [[0],
            [0, np.exp(ln_c11), 0], 
            [0, 0, 0, 0, 0],
            [0, -np.exp(ln_c13), c23, c33, 0, 0, 0, 0]]
    model = Model(hotspot_offset=offset, 
                  omega_drag=np.exp(ln_omega_drag),
                  alpha=alpha, C_ml=C_ml, lmax=3, A_B=0,
                  a_rs=p.a, rp_a=p.rp_a, T_s=p.T_s, filt=filt)

    xi = np.linspace(-2.7, 2.7, 20)
    phase_curve = model.phase_curve(xi)

    plt.plot(xi, 1e3 * phase_curve)
    plt.xlabel('$\\xi/\\pi$')
    plt.ylabel('Flux [ppt]')
    plt.show()